In [12]:
import pandas as pd
#  import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import numpy as np

In [13]:
from tensorflow.keras.models import model_from_json

json_file = open('modelepoch1.json', 'r')
loaded_model_json = json_file.read()

json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
# loaded_model.load_weights("modelepoch1.weights.h5")
print("Loaded model from disk")
loaded_model.summary()
 
# evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1800, 32)       │     6,400,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,000,014 (49.59 MB)

 Trainable params: 6,500,006 (24.80 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,500,008 (24.80 MB)

In [14]:
MAX_FEATURES=200000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [15]:
# vectorized_comment = vectorizer.adapt(["Inspiration comes from within yourself. One has to be positive. When you are positive, good things happen.."])
# print(vectorized_comment)

In [25]:
# Fit the state of the preprocessing layer to the dataset
comment="you are such a great person"

vectorizer.adapt([comment])

# Now, transform your text into a numerical representation
vectorized_comment = vectorizer([comment])

print(vectorized_comment)

tf.Tensor([[2 6 3 ... 0 0 0]], shape=(1, 1800), dtype=int64)


In [17]:
col_list=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']

In [27]:
def score_comment(comment):
    vectorizer.adapt([comment])
    vectorized_comment = vectorizer(comment)
    print(vectorized_comment)
    res = loaded_model.predict(np.expand_dims(vectorized_comment,0))
    # results = loaded_model.predict(vectorized_comment)
    # print(res)
    text = ''
    for idx, col in enumerate(col_list):
        text += '{}: {}\n'.format(col, res[0][idx]>0.5)
    print(text)
    new_df = pd.DataFrame(columns=col_list, data=res)
    print(new_df.head)
    return res

In [31]:
score_comment("I love you")

tf.Tensor([4 3 2 ... 0 0 0], shape=(1800,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
toxic: False
severe_toxic: False
obscene: True
threat: True
insult: False
identity_hate: True

<bound method NDFrame.head of       toxic  severe_toxic   obscene    threat    insult  identity_hate
0  0.496489      0.499779  0.500311  0.501903  0.498859        0.50102>


array([[0.49648854, 0.4997787 , 0.50031143, 0.50190294, 0.49885923,
        0.5010197 ]], dtype=float32)